In [30]:
start_lon=str(72.831353)
start_lat=str(18.968835)
end_lon=str(77.166284)
end_lat=str(28.677697)

In [31]:
import requests
r = requests.get('http://0.0.0.0:5000/route/v1/driving/'+start_lon+','+start_lat+';'+end_lon+','+end_lat+'?alternatives=3&overview=false&steps=true')
route_data=r.json()


In [32]:
def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    return c * r

In [38]:
num_routes=len(route_data['routes'])
route_data['routes'][0]['legs'][0]['steps'][0]['intersections'][0]['location'][1]

18.968354

In [ ]:
def plan_route_oneway(route_json,prev_lon,_prev_lat):
    #assuming only 1 leg
    range_left=range_car
    route_array=[]
    num_steps=len(route_json['legs'][0]['steps'])
    for i in range(0,num_steps):
        num_intersections=len(route_json['legs'][0]['steps'][i]['intersection'])
        for j in range(0,num_intersections):
            offset=0#Have to put offset ehere
            lat_new=route_json['legs'][0]['steps'][i]['intersection'][j]['location'][1]
            lon_new=route_json['legs'][0]['steps'][i]['intersection'][j]['location'][0]
            range_left=range_left-(haversine(prev_lon,prev_lat,lon_new,lat_new)+offset)
            if(stations_present):
                range_left=range_car
                last_charge_lat=lat_new
                
            prev_lat=lat_new
            prev_lon=lon_new
            route_array.append(route_json['legs'][0]['steps'][i]['intersection'][j]['location'])
            if(range_left<=0)
                return []

In [ ]:
def plan_route(start_lon,start_lat,end_lon,end_lat):
    r = requests.get('http://0.0.0.0:5000/route/v1/driving/'+start_lon+','+start_lat+';'+end_lon+','+end_lat+'?alternatives=3&overview=false&steps=true')
    route_data=r.json()
    num_routes=len(route_data['routes'])
    route_arr=[]
    for i in range(0,num_routes):
        route_arr.append([])
        route_arr[i]+plan_route_oneway(route_data['routes'][i],prev_lon,prev_lat)
    return route_arr    
    